读入tips.csv 数据集

1. 统计不同time的tip的均值，方差

2. 将total_bill和tip根据不同的sex进行标准化(原数据减去均值的结果除以标准差)

3. 计算吸烟者和非吸烟者的小费比例值均值  的差值

4. 对sex与size聚合，统计不同分组的小费比例的标准差、均值，将该标准差与均值添加到原数据中

5. 对time和size聚合，画出total_bill 的饼图

In [1]:
from __future__ import division
from numpy.random import randn
import numpy as np
import os
import matplotlib.pyplot as plt
from pandas import Series, DataFrame
import pandas as pd
import matplotlib as mpl
import matplotlib.dates as mdates
from mpl_toolkits.mplot3d import Axes3D
import numpy as np



In [2]:
import pathlib as PL
home = PL.Path(get_ipython().magic(u'pwd'))
tips = pd.read_csv(home/'data/tips.csv')
tips.head()

,total_bill,tip,sex,smoker,day,time,size
0,16.99,1.01,Female,No,Sun,Dinner,2
1,10.34,1.66,Male,No,Sun,Dinner,3
2,21.01,3.50,Male,No,Sun,Dinner,3
3,23.68,3.31,Male,No,Sun,Dinner,2
4,24.59,3.61,Female,No,Sun,Dinner,4


# 1.统计不同time的tip的均值，方差

In [3]:
tips.groupby('time').describe()

size                                                 tip            \
        count      mean       std  min  25%  50%  75%  max  count      mean   
time                                                                          
Dinner  176.0  2.630682  0.910241  1.0  2.0  2.0  3.0  6.0  176.0  3.102670   
Lunch    68.0  2.411765  1.040024  1.0  2.0  2.0  2.0  6.0   68.0  2.728088   

        ...                 total_bill                                      \
        ...       75%   max      count       mean       std   min      25%   
time    ...                                                                  
Dinner  ...    3.6875  10.0      176.0  20.797159  9.142029  3.07  14.4375   
Lunch   ...    3.2875   6.7       68.0  17.168676  7.713882  7.51  12.2350   

                                
           50%      75%    max  
time                            
Dinner  18.390  25.2825  50.81  
Lunch   15.965  19.5325  43.11  

[2 rows x 24 columns]

In [4]:
# or
tips.groupby('time').agg(['mean', 'std'])

total_bill                 tip                size          
             mean       std      mean       std      mean       std
time                                                               
Dinner  20.797159  9.142029  3.102670  1.436243  2.630682  0.910241
Lunch   17.168676  7.713882  2.728088  1.205345  2.411765  1.040024

# 2.将total_bill和tip根据不同的sex进行标准化
(原数据减去均值的结果除以标准差)

In [5]:
t = tips.groupby('sex').agg(['mean', 'std'])
t 

total_bill                 tip                size          
             mean       std      mean       std      mean       std
sex                                                                
Female  18.056897  8.009209  2.833448  1.159495  2.459770  0.937644
Male    20.744076  9.246469  3.089618  1.489102  2.630573  0.955997

In [6]:
tb_m_F = t['total_bill']['mean'].Female
tb_m_M= t['total_bill']['mean'].Male
tb_std_F= t['total_bill']['std'].Female
tb_std_M= t['total_bill']['std'].Male

t_m_F = t['tip']['mean'].Female
t_m_M= t['tip']['mean'].Male
t_std_F= t['tip']['std'].Female
t_std_M= t['tip']['std'].Male



In [7]:
tips['new_total_bill'] = tips['total_bill'] / 1
tips['new_tip'] = tips['tip'] / 1


In [8]:
for i in range(len(tips["total_bill"])):
    if tips['sex'][i] == "Male":
        tips["new_total_bill"][i] = (tips["total_bill"][i] - tb_m_M) / tb_std_M
        tips['new_tip'][i] = (tips['tip'][i] - t_m_M) / t_std_M
    else:
        tips["new_total_bill"][i] = (tips["total_bill"][i] - tb_m_F) / tb_std_F
        tips['new_tip'][i] = (tips['tip'][i] - t_m_F) / t_std_F

tips.head()

C:\Anaconda3\lib\site-packages\ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
C:\Anaconda3\lib\site-packages\ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  import sys
C:\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until
C:\Anaconda3\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is 

,total_bill,tip,sex,smoker,day,time,size,new_total_bill,new_tip
0,16.99,1.01,Female,No,Sun,Dinner,2,-0.133209,-1.572623
1,10.34,1.66,Male,No,Sun,Dinner,3,-1.125195,-0.960054
2,21.01,3.50,Male,No,Sun,Dinner,3,0.028759,0.275590
3,23.68,3.31,Male,No,Sun,Dinner,2,0.317518,0.147997
4,24.59,3.61,Female,No,Sun,Dinner,4,0.815699,0.669733


# 3.计算吸烟者和非吸烟者的小费比例值均值 的差值

理解一：先求吸烟与不吸烟人群的小费平均值，再求吸烟与不吸烟人群的账单值，最后再求百分比差值。

In [9]:
t = tips.groupby('smoker')
tb = t['total_bill']
tt = t['tip']

no = (tt.mean().No / tb.mean().No )* 100
yes = (tt.mean().Yes / tb.mean().Yes )* 100

In [10]:
print("不吸烟人群中，小费均值与账单均值的百分比：",str(no),"%")
print("吸烟人群中，小费均值与账单均值的百分比：",str(yes),"%")
print("不吸烟与吸烟的消费比例均值的差值：",str(no - yes),"%")

不吸烟人群中，小费均值与账单均值的百分比： 15.59209368302254 %
吸烟人群中，小费均值与账单均值的百分比： 14.495373871960378 %
不吸烟与吸烟的消费比例均值的差值： 1.0967198110621617 %


理解二：求每一行的tip/total_bill的比例值均值，再求差

In [11]:
tips['tip_pct'] = tips['tip'] / tips['total_bill']
tips[:6]

grouped = tips.groupby(['smoker'])

grouped_pct = grouped['tip_pct']
g = grouped_pct.agg('mean')

In [12]:
print("不吸烟人群中，小费均值与账单均值的百分比：",str(g.No*100),"%")
print("吸烟人群中，小费均值与账单均值的百分比：",str(g.Yes*100),"%")
print("不吸烟与吸烟的消费比例均值的差值：",str((g.No - g.Yes)*100),"%")

不吸烟人群中，小费均值与账单均值的百分比： 15.932846217921531 %
吸烟人群中，小费均值与账单均值的百分比： 16.31960446368779 %
不吸烟与吸烟的消费比例均值的差值： -0.3867582457662583 %


# 4.对sex与size聚合，统计不同分组的小费比例的标准差、均值，将该标准差与均值添加到原数据中

In [98]:
t = tips.groupby(['sex', 'size'])

fun = ['std', 'mean']
new_cols = t['tip'].agg(fun)
# new_cols = new_cols.unstack(-2)
new_cols


sex     size
Female  1       0.479201
        2       0.882510
        3       1.291398
        4       1.084924
        5            NaN
        6       0.565685
Male    1            NaN
        2       1.044757
        3       1.714419
        4       1.798484
        5       1.500000
        6       1.202082
Name: tip, dtype: float64

In [14]:
a = list()
b = list(range(1,7)) * 2
for i in range(0,12):
    s = b[i]
    a.append(str(s))
    
a

['1', '2', '3', '4', '5', '6', '1', '2', '3', '4', '5', '6']

In [93]:
new = DataFrame({
    "size": a,
    "sex": list(["Female"]) * 6 + list(["Male"]) *6,
    "std": np.array(new_cols['std']),
    "mean": np.array(new_cols['mean'])},
    columns = ["sex","size","mean","std"]
)

new.head()

sex      size      mean       std
size sex                                         
1    Female  Female  0.215131  1.276667  0.479201
     Male    Female  0.223776  2.528448  0.882510
2    Female  Female  0.170830  3.250000  1.291398
     Male    Female  0.162694  4.021111  1.084924
3    Female  Female  0.159899  5.140000       NaN

In [107]:
a = tips.groupby(["size","sex"]).mean().mean_tip_pct
a = DataFrame(a)
a.head()

mean_tip_pct
size sex                 
1    Female      0.215131
     Male        0.223776
2    Female      0.170830
     Male        0.162694
3    Female      0.159899

In [109]:
b = tips.groupby(["size","sex"]).describe().tip_pct["std"]
b = DataFrame(b)
b.head()

std
size sex             
1    Female  0.098256
     Male         NaN
2    Female  0.055211
     Male    0.072970
3    Female  0.049302

In [126]:
new = a.join(b)
new

mean_tip_pct       std
size sex                           
1    Female      0.215131  0.098256
     Male        0.223776       NaN
2    Female      0.170830  0.055211
     Male        0.162694  0.072970
3    Female      0.159899  0.049302
     Male        0.147641  0.043506
4    Female      0.132734  0.018139
     Male        0.150197  0.047139
5    Female      0.172194       NaN
     Male        0.133821  0.075659
6    Female      0.162891  0.031044
     Male        0.149567  0.064726

In [124]:
tips.join(new)
new.head()

ValueError: columns overlap but no suffix specified: Index(['sex', 'size', 'mean_tip_pct'], dtype='object')

In [21]:
pd.merge(tips, new, left_on='size', right_index=True)


ValueError: len(left_on) must equal the number of levels in the index of "right"